In [1]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from sklearn.model_selection import KFold
import os, json, cv2, random, sys
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.utils import comm
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

import argparse

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/M6-T6-Project-master/week3

/content/drive/MyDrive/M6-T6-Project-master/week3


In [5]:
#!python inference_taska.py

In [6]:
#!pip install xmltodict
sys.path.append('../week1')
from utils_week1 import read_annotations
#import bounding_box

#from LossEvalHook import *
#from MyTrainerAugm import *

In [7]:
def parse_annotation(annotations):
    objs = []
    for annot in annotations:

        bbox = [annot[0], annot[1], annot[2], annot[3]]

        obj = {
            "bbox": bbox,
            "bbox_mode": BoxMode.XYXY_ABS,
            "category_id": 0
        }

        objs.append(obj)

    return objs

In [8]:
def load_dataset(type, thing_classes):

    frames_path = '../data/images'
    gt_path = '../data/ai_challenge_s03_c010-full_annotation.xml'

    gt = read_annotations(gt_path, use_parked=True)

    num_frames = 2141

    frame_list = np.arange(0,2141)

    train_val_split = 0.67
    # train_val_split = 1.0

    if type == 'train':
        init_frame = 0
        last_frame = int(train_val_split*(int(num_frames*0.25)))
    elif type == 'val' or type == 'test':
        init_frame = int(train_val_split*(int(num_frames*0.25)))
        last_frame = int(num_frames*0.25)
    # elif type == 'test':
    #     init_frame = int(num_frames*0.25)
    #     last_frame = num_frames
    
    dataset_dicts = []
    for frame_id in range(init_frame, last_frame):
        record = {}
        filename = os.path.join(frames_path, "%04d.jpeg" % (frame_id+1))

        record["file_name"] = filename
        # record["image_id"] = i+j
        record["image_id"] = filename
        record["height"] = 1080
        record["width"] = 1920

        record["annotations"] = parse_annotation(gt[frame_id])
        dataset_dicts.append(record)

    return dataset_dicts

In [9]:
# ValidationLoss (monitor the validation loss)
class ValidationLoss(HookBase):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()
        self.cfg.DATASETS.TRAIN = cfg.DATASETS.TEST
        self._loader = iter(build_detection_train_loader(self.cfg))

    def after_step(self):
        data = next(self._loader)
        with torch.no_grad():
            loss_dict = self.trainer.model(data)

            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {"val_" + k: v.item() for k, v in
                                 comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                self.trainer.storage.put_scalars(total_val_loss=losses_reduced,
                                                 **loss_dict_reduced)


In [10]:
#args = parse_args()

model = 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'  #pre-trained detectron2 model
print('[INFO] Using model: ', model)

#Hyper-parameters
lr= 0.001      #learning rate
epochs= 100    #max iterations (epochs)
batch= 512     #batch size
#augm=True     #use augmentation
#freeze=2      #stages of ResNet to freeze


###-------TRAIN-----------------------------
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)

cfg.OUTPUT_DIR = 'results/task1_2/' + 'faster_rcnn' + '/lr_' + str(lr).replace('.', '_') + '_iter_' + str(epochs) + '_batch_' + str(batch) + '/' 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

thing_classes = ['Car']
dataset='AICity'

for d in ['train', 'val', 'test']:
  DatasetCatalog.register(dataset + '_' + d, lambda d=d: load_dataset(d, thing_classes))
  MetadataCatalog.get(dataset + '_' + d).set(thing_classes=thing_classes)

metadata = MetadataCatalog.get(dataset + '_train')

cfg.DATASETS.TRAIN = (dataset + '_train',)
cfg.DATASETS.VAL = (dataset + '_val',)
cfg.DATASETS.TEST = (dataset + '_test',)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)
cfg.SOLVER.IMS_PER_BATCH = 2

#cfg.MODEL.BACKBONE.FREEZE_AT = args.freeze

cfg.SOLVER.BASE_LR = lr
cfg.SOLVER.MAX_ITER = epochs
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = batch

#eval_period: frequence of validation loss computations (to plot curves)
cfg.TEST.EVAL_PERIOD = 0

#if args.augm:
#    trainer = MyTrainerAugm(cfg)
#else:
#    trainer = MyTrainer(cfg)

trainer = DefaultTrainer(cfg)
val_loss = ValidationLoss(cfg)
trainer.register_hooks([val_loss])
# swap the order of PeriodicWriter and ValidationLossl
trainer._hooks = trainer._hooks[:-2] + trainer._hooks[-2:][::-1]
trainer.resume_or_load(resume=False)
trainer.train()


###-------INFERENCE AND EVALUATION---------------------------
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

### MAP #####
#We can also evaluate its performance using AP metric implemented in COCO API.
evaluator = COCOEvaluator(dataset + '_val', cfg, False, output_dir=cfg.OUTPUT_DIR)
test_loader = build_detection_test_loader(cfg, dataset + '_val')
print('------------------------ Evaluating model ' + model + ' on validation set ---------------------------------')
print(inference_on_dataset(trainer.model, test_loader, evaluator))
print('---------------------------------------------------------')




[INFO] Using model:  COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
[03/28 23:51:00 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBat

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[03/28 23:51:09 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/28 23:51:53 d2.utils.events]:  eta: 0:01:56  iter: 19  total_loss: 1.554  loss_cls: 0.639  loss_box_reg: 0.8514  loss_rpn_cls: 0.02584  loss_rpn_loc: 0.03587  total_val_loss: 1.565  val_loss_cls: 0.6203  val_loss_box_reg: 0.8712  val_loss_rpn_cls: 0.02285  val_loss_rpn_loc: 0.04501  time: 1.4157  data_time: 0.0785  lr: 0.00019081  max_mem: 2639M
[03/28 23:52:34 d2.utils.events]:  eta: 0:01:27  iter: 39  total_loss: 1.26  loss_cls: 0.38  loss_box_reg: 0.8352  loss_rpn_cls: 0.01476  loss_rpn_loc: 0.03221  total_val_loss: 1.289  val_loss_cls: 0.3843  val_loss_box_reg: 0.8519  val_loss_rpn_cls: 0.01633  val_loss_rpn_loc: 0.03858  time: 1.4342  data_time: 0.0073  lr: 0.00039061  max_mem: 2639M
[03/28 23:53:15 d2.utils.events]:  eta: 0:00:57  iter: 59  total_loss: 1.079  loss_cls: 0.2362  loss_box_reg: 0.797  loss_rpn_cls: 0.009985  loss_rpn_loc: 0.02417  total_val_loss: 1.115  val_loss_cls: 0.282  val_loss_box_reg: 0.7856  val_loss_rpn_cls: 0.01049  val_loss_rpn_loc: 0.03177  time: 1.42